In [2]:
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders
from datetime import datetime

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
all_folders_drop = get_dropbox_folders("/DATA/PRINCE", True)

2022-09-16 17:30:16,092-[INFO]- dropbox:388 -> Refreshing access token.
2022-09-16 17:30:16,289-[INFO]- dropbox:477 -> Request to files/list_folder
2022-09-16 17:30:17,743-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-09-16 17:30:18,610-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-09-16 17:30:19,506-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-09-16 17:30:20,366-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-09-16 17:30:21,279-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-09-16 17:30:22,571-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-09-16 17:30:23,663-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-09-16 17:30:24,635-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-09-16 17:30:25,688-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-09-16 17:30:26,552-[INFO]- dropbox:477 -> Request to files/list_folder/continue
20

In [8]:
on_dropbox = all_folders_drop.loc[
    all_folders_drop["CrossDate"].between("20220801", "20220820")
]
plates = on_dropbox["unique_id"].unique()

In [14]:
all_folders_drop.loc[all_folders_drop["folder"].str.contains("Analysis")]

,folder,size,change_date,tot_path_drop,Plate,CrossDate,unique_id
299,Analysis_1045_20220504_200_Version1,NaN,2022-09-12 22:09:48,data/prince/1045_20220504/analysis_1045_202205...,1045,20220504,1045_20220504
300,Analysis_1048_20220506_200_Version1,NaN,2022-09-12 22:09:19,data/prince/1048_20220506/analysis_1048_202205...,1048,20220506,1048_20220506
301,Analysis_792_20210915_104_Version1,NaN,2022-09-12 20:52:56,data/prince/792_20210915/analysis_792_20210915...,792,20210915,792_20210915
302,Analysis_152_20201224_200_Version1,NaN,2022-09-12 20:52:35,data/prince/152_20201224/analysis_152_20201224...,152,20201224,152_20201224
303,Analysis_76_20210108_200_Version1,NaN,2022-09-12 20:51:33,data/prince/76_20210108/analysis_76_20210108_2...,76,20210108,76_20210108
...,...,...,...,...,...,...,...
17020,Analysis_4_20220419_38_Version1,NaN,2022-06-29 07:37:33,data/prince/4_20220419/analysis_4_20220419_38_...,4,20220419,4_20220419
17021,Analysis_4_20220419_64_Version1,NaN,2022-06-29 07:37:21,data/prince/4_20220419/analysis_4_20220419_64_...,4,20220419,4_20220419
17022,Analysis_4_20220419_62_Version1,NaN,2022-06-29 07:37:08,data/prince/4_20220419/analysis_4_20220419_62_...,4,20220419,4_20220419
17190,Analysis_1053_20220413_85_Version1,NaN,2022-06-29 03:29:27,data/prince/1053_20220413/analysis_1053_202204...,1053,20220413,1053_20220413


In [56]:
on_dropbox = all_folders_drop.loc[all_folders_drop["unique_id"].isin(plates)]

In [8]:
on_dropbox = all_folders_drop.loc[all_folders_drop["unique_id"].isin(plates[5:])]
len(on_dropbox)

1771

In [1]:
# plates= ['94_20201123','102_20201226','76_20210108','152_20201224','792_20210915']
# plates= ['52_20220707','56_20220701','839_20220325','938_20220325','1076_20220511']
plates = ["12_20220502"]
# plates = ['52_20220707','1048_20220506','1045_20220504']

# plates = ['1030_20220523','23_20220509','24_20220426','33_20220502']
# plates = ["23_20220509"]

In [6]:
plates = "19_20220525,20_20220419,22_20220522,23_20220509,24_20220426,27_20220508,28_20220603,30_20220509,31_20220428,32_20220428,33_20220502"
plates = plates.split(",")

In [35]:
# On project
plates = "127_20220705,128_20220718,129_20220711,130_20220707,1014_20220525,1015_20220504,1023_20220502,1027_20220607,1028_20220523,1037_20220523,"
plates = plates.split(",")

In [50]:
all_folders_drop["folder"].str.split("_", expand=True)[0]

0        20200629
1        20200629
2        20200629
3        20200629
4        20200629
           ...   
41264    20201123
41265    20201123
41266    20201123
41267    20201123
41268    20201123
Name: 0, Length: 41269, dtype: object

In [40]:
select_plates = [
    plate
    for plate in all_folders_drop["unique_id"].unique()
    if plate.split("_")[1] == "20220623"
]

In [52]:
folders_drop = all_folders_drop.loc[
    all_folders_drop["unique_id"].isin(select_plates)
].loc[all_folders_drop["folder"].str.split("_", expand=True)[0] == "20220809"]

In [3]:
directory_targ = directory_project
directory_project

'/projects/0/einf914/data/'

In [4]:
directory_targ = os.path.join(directory_scratch, "stitch_temp2") + "/"
# plates = ['1053_20220413']

In [8]:
name_job = f"one_shot_{time_ns()}.sh"
run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, -1,"stitcher.py"],
    plates,
    "12:00:00",
    name_job=name_job,
)
# run_launcher('dropbox_uploader.py',[directory_targ,name_job],plates,'40:00',dependency=True,name_job = name_job)

Submitted batch job 1565150


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


In [9]:
name_job = f"one_shot_{time_ns()}.sh"
run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, 10,"hypha_identifier.py"],
    plates,
    "12:00:00",
    name_job=name_job,
)

Submitted batch job 1566244


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


In [61]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Img/TileConfiguration.txt.registered"] == True]
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
10,/projects/0/einf914/data/20220601_1143_Plate12,30,12,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"01.06.2022, 11:43:",20220601_1143_Plate12,30_20220509,2022-06-01 11:43:00
23,/projects/0/einf914/data/20220618_0150_Plate11,23,11,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"18.06.2022, 01:50:",20220618_0150_Plate11,23_20220509,2022-06-18 01:50:00
24,/projects/0/einf914/data/20220528_1900_Plate12,30,12,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"28.05.2022, 19:00:",20220528_1900_Plate12,30_20220509,2022-05-28 19:00:00
25,/projects/0/einf914/data/20220619_0156_Plate11,23,11,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"19.06.2022, 01:56:",20220619_0156_Plate11,23_20220509,2022-06-19 01:56:00
29,/projects/0/einf914/data/20220613_0147_Plate12,30,12,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.06.2022, 01:47:",20220613_0147_Plate12,30_20220509,2022-06-13 01:47:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4882,/projects/0/einf914/data/20220529_0302_Plate12,30,12,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"29.05.2022, 03:02:",20220529_0302_Plate12,30_20220509,2022-05-29 03:02:00
4886,/projects/0/einf914/data/20220604_0131_Plate11,23,11,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"04.06.2022, 01:31:",20220604_0131_Plate11,23_20220509,2022-06-04 01:31:00
4889,/projects/0/einf914/data/20220619_1955_Plate11,23,11,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"19.06.2022, 19:55:",20220619_1955_Plate11,23_20220509,2022-06-19 19:55:00
4890,/projects/0/einf914/data/20220616_1904_Plate10,22,10,'Carrot','Agg','001P100N100C','Y','-',20220522,25,...,False,False,False,False,False,False,"16.06.2022, 19:04:",20220616_1904_Plate10,22_20220522,2022-06-16 19:04:00


In [4]:
run_launcher("skeletonizer.py", [directory_targ], plates, "20:00", dependency=True)

Submitted batch job 1129540


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 245760 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


In [20]:
name_job = f"one_shot_{time_ns()}.sh"

run_launcher(
    "skeletonizer.py",
    [directory_targ, name_job, 30],
    plates,
    "12:00:00",
    dependency=True,
    name_job=name_job,
)

Submitted batch job 1391860


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 245760 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


In [4]:
name_job = f"one_shot_{time_ns()}.sh"
run_launcher(
    "hypha_identifier.py",
    [directory_targ, name_job, 10],
    plates,
    "20:00",
    dependency=True,
    name_job=name_job,
)

Submitted batch job 1559631


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 245760 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


In [4]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

analysed:   0%|          | 0/2691 [00:00<?, ?it/s]

In [5]:
# plates= ['12_20220502']
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
print(len(folders))
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
print(len(folders))

# folders = folders.loc[folders['/Analysis/nx_graph_pruned_labeled.p']==True]
# folders = folders.loc[folders['/Analysis/skeleton_compressed.mat']==False]
plates_2 = folders["unique_id"].unique()

648
382


In [9]:
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
13,/scratch-shared/amftrack/stitch_temp2/20220515...,23,11,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"15.05.2022, 22:18:",20220515_2218_Plate11,23_20220509,2022-05-15 22:18:00
20,/scratch-shared/amftrack/stitch_temp2/20220513...,23,11,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"13.05.2022, 08:17:",20220513_0817_Plate11,23_20220509,2022-05-13 08:17:00
24,/scratch-shared/amftrack/stitch_temp2/20220517...,33,7,'Carrot','Agg','001P100N100C','Y','-',20220502,25,...,False,False,False,False,False,False,"17.05.2022, 17:05:",20220517_1705_Plate07,33_20220502,2022-05-17 17:05:00
35,/scratch-shared/amftrack/stitch_temp2/20220517...,23,11,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"17.05.2022, 15:17:",20220517_1517_Plate11,23_20220509,2022-05-17 15:17:00
50,/scratch-shared/amftrack/stitch_temp2/20220610...,23,11,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"10.06.2022, 23:44:",20220610_2344_Plate11,23_20220509,2022-06-10 23:44:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006,/scratch-shared/amftrack/stitch_temp2/20220511...,23,11,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"11.05.2022, 13:11:",20220511_1311_Plate11,23_20220509,2022-05-11 13:11:00
2031,/scratch-shared/amftrack/stitch_temp2/20220514...,23,11,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"14.05.2022, 00:16:",20220514_0016_Plate11,23_20220509,2022-05-14 00:16:00
2034,/scratch-shared/amftrack/stitch_temp2/20220510...,33,7,'Carrot','Agg','001P100N100C','Y','-',20220502,25,...,False,False,False,False,False,False,"10.05.2022, 09:59:",20220510_0959_Plate07,33_20220502,2022-05-10 09:59:00
2038,/scratch-shared/amftrack/stitch_temp2/20220614...,23,11,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,False,"14.06.2022, 07:03:",20220614_0703_Plate11,23_20220509,2022-06-14 07:03:00


In [7]:
import sys
from amftrack.util.sys import (
    update_plate_info,
    get_current_folders,
)
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Analysis/skeleton_realigned_compressed.mat"] == True]
num_parallel = 128
time = "12:00:00"
skip = False
resolution = 10
args = [directory_targ, skip, resolution]
run_parallel(
    "extract_width.py",
    args,
    folders,
    num_parallel,
    time,
    "extract_width2",
    cpus=32,
    node="fat",
)

Submitted batch job 1416009


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1416010
Submitted batch job 1416011
Submitted batch job 1416012
Submitted batch job 1416013


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [11]:
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    run_launcher,
    run_parallel_all_time,
)

# for unique_id in plates:
#     select = folders.loc[folders["unique_id"] == unique_id]
#     time = "2:00:00"
#     run_parallel(
#         "track_nodes.py",
#         args,
#         select,
#         num_parallel,
#         time,
#         "track_node",
#         cpus=128,
#         node="fat",
#         dependency=False,
#         # name_job='job.sh'
#     )

time = "12:00:00"
run_parallel_all_time(
    "make_labeled_graphs.py",
    args,
    folders,
    num_parallel,
    time,
    "make_graphs",
    cpus=128,
    node="fat",
    dependency=True,
    name_job="job.sh",
)

Submitted batch job 1416297


In [23]:
plates_2

array(['130_20220707', '1042_20220523', '1030_20220523'], dtype=object)

In [13]:
i = 0
unique_ids = list(set(folders["unique_id"].values))
unique_ids.sort()
select = folders.loc[folders["unique_id"] == unique_ids[i]]

In [6]:
all_folders
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
17,/scratch-shared/amftrack/stitch_temp2/20220709...,510,11,'None','C2','Myr','Y','-',20220623,25,...,True,False,True,True,True,True,"09.07.2022, 02:00:",20220709_0200_Plate11,510_20220623,2022-07-09 02:00:00
32,/scratch-shared/amftrack/stitch_temp2/20220702...,510,11,'None','C2','Myr','Y','-',20220623,25,...,True,False,True,True,True,True,"02.07.2022, 17:39:",20220702_1739_Plate11,510_20220623,2022-07-02 17:39:00
40,/scratch-shared/amftrack/stitch_temp2/20220701...,510,11,'None','C2','Myr','Y','-',20220623,25,...,True,False,True,True,True,True,"01.07.2022, 11:39:",20220701_1139_Plate11,510_20220623,2022-07-01 11:39:00
61,/scratch-shared/amftrack/stitch_temp2/20220708...,510,11,'None','C2','Myr','Y','-',20220623,25,...,True,False,True,True,True,True,"08.07.2022, 17:59:",20220708_1759_Plate11,510_20220623,2022-07-08 17:59:00
69,/scratch-shared/amftrack/stitch_temp2/20220624...,510,11,'None','C2','Myr','Y','-',20220623,25,...,True,False,True,True,True,True,"24.06.2022, 20:36:",20220624_2036_Plate11,510_20220623,2022-06-24 20:36:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,/scratch-shared/amftrack/stitch_temp2/20220704...,510,11,'None','C2','Myr','Y','-',20220623,25,...,True,False,True,True,True,True,"04.07.2022, 05:40:",20220704_0540_Plate11,510_20220623,2022-07-04 05:40:00
1568,/scratch-shared/amftrack/stitch_temp2/20220717...,510,11,'None','C2','Myr','Y','-',20220623,25,...,True,False,False,False,False,False,"17.07.2022, 23:11:",20220717_2311_Plate11,510_20220623,2022-07-17 23:11:00
1574,/scratch-shared/amftrack/stitch_temp2/20220708...,510,11,'None','C2','Myr','Y','-',20220623,25,...,True,False,True,True,True,True,"08.07.2022, 11:21:",20220708_1121_Plate11,510_20220623,2022-07-08 11:21:00
1576,/scratch-shared/amftrack/stitch_temp2/20220717...,510,11,'None','C2','Myr','Y','-',20220623,25,...,True,False,False,False,False,False,"17.07.2022, 01:11:",20220717_0111_Plate11,510_20220623,2022-07-17 01:11:00


In [4]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
# folders = folders.loc[folders["/Analysis/skeleton_pruned_compressed.mat"]==True]
folders

analysed:   0%|          | 0/3031 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [22]:
plates = all_folders.loc[
    all_folders["unique_id"].str.split("_", expand=True)[1] == "20220623"
]["unique_id"].unique()

In [23]:
plates

array(['507_20220623', '510_20220623', '509_20220623', '508_20220623',
       '502_20220623', '515_20220623'], dtype=object)

In [8]:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders.loc[folders["/Img/TileConfiguration.txt.registered"] == True]

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
3,/scratch-shared/amftrack/stitch_temp/20220407_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"07.04.2022, 21:55:",20220407_2155_Plate33,889_20220325,2022-04-07 21:55:00
18,/scratch-shared/amftrack/stitch_temp/20220327_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"27.03.2022, 15:35:",20220327_1535_Plate14,889_20220325,2022-03-27 15:35:00
30,/scratch-shared/amftrack/stitch_temp/20220330_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"30.03.2022, 21:40:",20220330_2140_Plate14,889_20220325,2022-03-30 21:40:00
39,/scratch-shared/amftrack/stitch_temp/20220419_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"19.04.2022, 01:20:",20220419_0120_Plate33,889_20220325,2022-04-19 01:20:00
62,/scratch-shared/amftrack/stitch_temp/20220328_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"28.03.2022, 05:36:",20220328_0536_Plate14,889_20220325,2022-03-28 05:36:00
64,/scratch-shared/amftrack/stitch_temp/20220415_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"15.04.2022, 05:55:",20220415_0555_Plate33,889_20220325,2022-04-15 05:55:00
86,/scratch-shared/amftrack/stitch_temp/20220401_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"01.04.2022, 18:45:",20220401_1845_Plate14,889_20220325,2022-04-01 18:45:00
98,/scratch-shared/amftrack/stitch_temp/20220330_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"30.03.2022, 23:40:",20220330_2340_Plate14,889_20220325,2022-03-30 23:40:00
101,/scratch-shared/amftrack/stitch_temp/20220329_...,889,14,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"29.03.2022, 21:10:",20220329_2110_Plate14,889_20220325,2022-03-29 21:10:00
112,/scratch-shared/amftrack/stitch_temp/20220418_...,889,33,'Carrot','A5','001P100N100C','Y','-',20220325,25,...,False,False,False,False,False,False,"18.04.2022, 01:20:",20220418_0120_Plate33,889_20220325,2022-04-18 01:20:00


In [12]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

for plate in plates:
    select = folders.loc[folders["Plate"] == plate]
    num_parallel = 128
    time = "12:00:00"
    thresh = 10000  # For R. irregularis, thresh 10000 is good. For Aggregatum, higher may be necessary
    args = [thresh, directory_targ]
    run_parallel(
        "final_alignment.py",
        args,
        select,
        num_parallel,
        time,
        "realign",
        cpus=128,
        node="fat",
        name_job=name_job,
    )

Submitted batch job 1138954


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [20]:
plates

['6_20220509']

In [19]:
select

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime


In [13]:
i = 0
run_info = folders
folder_list = list(run_info["folder"])
folder_list.sort()

dates_datetime_chosen = folder_list[i : i + 2]
print("========")
print(f"Matching plate {dates_datetime_chosen[0]} at dates {dates_datetime_chosen}")
print("========")
dates = dates_datetime_chosen

Matching plate 20220510_1851_Plate17 at dates ['20220510_1851_Plate17', '20220511_1750_Plate17']


In [10]:
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
from amftrack.pipeline.functions.post_processing.exp_plot import *

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
directory = directory_targ

In [11]:
time = "3:40:00"
directory = directory_targ
# list_f = [get_width_f,get_tot_length_C_f,get_tot_growth_C_f]
# list_f = [get_timestep_anastomosis]
list_f = [
    gets_out_of_ROI,
    get_width_f,
    get_tot_length_C_f,
    get_tot_growth_C_f,
    get_tot_length_pp_f,
    get_tot_growth_pp_f,
    get_timestep_stop_growth,
    get_time_stop_growth,
    get_time_init_growth,
    get_mean_speed_growth,
    get_stop_track,
    get_timestep_anastomosis,
    get_timestep_biological_stop_growth,
]
# list_f = [get_num_branch]
# list_f = [gets_out_of_ROI]
list_args = [{}] * len(list_f)
overwrite = False
num_parallel = 32
run_parallel_post(
    "global_hypha_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "global_hypha_post_process",
    cpus=32,
    name_job="a",
    node="fat",
)

Submitted batch job 1539552


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [13]:
import sys
from amftrack.util.sys import (
    update_analysis_info,
    get_analysis_info,
)
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
import pandas as pd
import os

list_f = [
    get_pos_x,
    get_pos_y,
    get_time_since_begin_exp,
    get_distance_final_pos,
    get_timedelta,
    get_time_since_start,
    get_speed,
    get_timestep,
    get_timestep_init,
    get_time_init,
    get_degree,
    get_width_tip_edge,
    get_width_root_edge,
    get_width_average,
    has_reached_final_pos,
    in_ROI,
]
# list_f = [local_density,local_density,local_density]
# list_f = [get_time_since_begin_exp]
# list_f = [get_width_tip_edge, get_width_root_edge]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = False
load_graphs = True
num_parallel = 32
time = "12:00:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_hypha_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "time_hypha_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )

Submitted batch job 1539625
Submitted batch job 1539626
Submitted batch job 1539627


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memor

Submitted batch job 1539628
Submitted batch job 1539629
Submitted batch job 1539630


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1539631


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1539645
Submitted batch job 1539646


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1539647
Submitted batch job 1539648
Submitted batch job 1539649


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
